## Pulling Data with Tweepy

**By:** _Jordan McNea_ 

In [ ]:
import datetime
import tweepy

# I've put my API keys in a .py file called API_keys.py
from API_keys import api_key, api_key_secret, access_token, access_token_secret

In [ ]:
# Authenticate the Tweepy API
auth = tweepy.OAuthHandler(api_key,api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

## Grab follower IDs

I had the WNBA Finals on in the background while creating this Notebook, so I will be collecting followers from the Seattle Storm and Las Vegas Aces, the two finalists. Tweepy only allows users to grab 900 requests per 15 minutes. It'll grab the 900 requests quickly then wait 15 minutes, rather than slowly grab 900 requests over a 15 minute period. Before we start grabbing follower IDs, let's first just check how long it will take. To do this we'll grab the followers_count item from Tweepy. 

In [ ]:
# I'm putting the handles in a list to iterate through below
team_handles = ['seattlestorm', 'LVAces']


# This will iterate through each Twitter handle that we're collecting from
for screen_name in team_handles:
    
    # Tells Tweepy we want information on the handle we're collecting from
    # The next line specifies which information we want, which in this case is the number of followers 
    user = api.get_user(screen_name) 
    followers_count = user.followers_count

    # Let's see roughly how long it will take to grab all the follower IDs. 
    print(f'''
    @{screen_name} has {followers_count} followers. 
    That will take roughly {followers_count/(5000*60):.0f} hours and {followers_count/(5000):.2f} minutes
    ''')
    

It looks like there should only be one fifteen minute break. It'll grab all of the Storm's followers, then some of the Aces before sleeping for fifteen minutes. Let's run it and see how long it'll actually take.

In [ ]:
# This creates a dictionary containing a list for each Twitter handle we'll be grabbing follower IDs from
id_dict = {'seattlestorm' : [],
           'LVAces' : []}

# Grabs the time when we start making requests to the API
start_time = datetime.datetime.now()

# .keys() allows us to iterate through each key in the dictionary
for handle in id_dict.keys():
    
    # Each page contains 5,000 records, so since we know there are much more than 5,000 followers for both
    # the Storm and Aces, we must iterate through each of the pages in order to get all follower IDs
    # To grab the follower IDs, we will be using followers_ids
    for page in tweepy.Cursor(api.followers_ids,
                              # This is how we will get around the issue of not being able to grab all ids at once
                              # Once the rate limit is hit, we will be notified that we must wait 15 mins (900 secs)
                              wait_on_rate_limit=True, wait_on_rate_limit_notify=True, compression=True,
                              screen_name=handle).pages():

        # The page variable comes back as a list, so we have to use .extend rather than .append
        id_dict[handle].extend(page)
        

# Let's see how long it took to grab all follower IDs
end_time = datetime.datetime.now()
elapsed_time = end_time - start_time
print(elapsed_time)

Let's look at some ids we gathered.

In [ ]:
id_dict['seattlestorm'][:10]

You'll notice they are all numbers. This is because ids are different from screen names. To see the twitter handles we gathered, we'll have to use the scren_name feature.

In [ ]:
users = id_dict['seattlestorm'][:10]

for name in users:
    
    user = api.get_user(name)
    print(user.screen_name)

## Grab descriptions based on the followers IDs

That looks much better. We can get all sorts of information from the ID. We don't just want screen names though, that doesn't tell us much. Let's grab each screen name and their description and write it to a text file for each team account.

In [ ]:
headers = ['screen_name','description']

for team in id_dict.keys():
    
    # Descriptions with emoji or non-Roman letters can cause trouble. Encoding your .txt file in utf-8 will help
    with open(f'{team}_followers.txt','w', encoding='utf-8') as out_file:
        wf.write('\t'.join(headers) + '\n')

        for idx, ids in enumerate(id_dict[team]):
            
            # For accounts set to private, we won't be able to get the description unless we follow them
            # Putting in a try/except statement, we can get around this issue.
            try:
                user = api.get_user(ids)
                description = str(user.description).replace('\t',' ').replace('\n',' ')
                outline = [user.screen_name, user.description]
                
                out_file.write('\t'.join([str(item) for item in outline]) + '\n')
                
            except:
                continue
                
            if idx == 100:
                break
            

## Grabbing Tweets by search terms

Tweepy also lets users grab tweets based off of search terms. October 10th was World Mental Health Day, so let's look at tweets containing its official hashtag. Twitter search allows standard search operators (<a href="https://developer.twitter.com/en/docs/twitter-api/v1/rules-and-filtering/overview/standard-operators">read more here</a>). We only want Tweets that occurred on World Mental Health Day, hence the since and until operators, and I'm excluding retweets.


In [ ]:
# Note: the search API only goes back 7 days
date_start = datetime.date.today()
date_end = date_start - datetime.timedelta(days=2)

search_words = f'#WorldMentalHealthDay since:{date_start} until:{date_end} -filter:retweets'

# Notice the differences between searching tweets and users. 
for idx, item in enumerate(tweepy.Cursor(api.search,
                   # tweet_mode is defaulted to short, which only holds the first 140 characters of a Tweet.
                   tweet_mode='extended',
                   q=search_words,
                   lang='en').items()):
    
    # There's all sort of information you can get from Tweets
    # Find more tweet objects here: https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/overview/tweet-object
    print(item.user.screen_name)
    print(item.created_at)
    print(item.full_text)
    print('-'*40)
    
    if idx == 50:
        break
    

It's also possible to use this search feature to grab the mentions of a Twitter account. Mentions are any tweet where another user's handle is included (i.e. they are mentioned in the tweet).

In [ ]:
search_words = '@GovernorBullock -filter:retweets'


tweets_all = tweepy.Cursor(api.search,
                   tweet_mode='extended',
                   q=search_words,
                   lang='en').items()

# Put all the Tweet objects for a single Tweet into a tuple, and put all those into a list
tweets = [(tweet.full_text,tweet.created_at,tweet.user.screen_name) for tweet in tweets_all]


In [ ]:
tweets[:10]